In [1]:
import googlemaps
import requests
import os
from dotenv import load_dotenv

load_dotenv()
gmaps = googlemaps.Client(key=os.getenv("GOOGLE_KEY"))


def geocoords(place_id):
    place_details = gmaps.place(place_id)
    latitude = place_details['result']['geometry']['location']['lat']
    longitude = place_details['result']['geometry']['location']['lng']
    return {'lat': latitude, 'lon': longitude}


def get_gmaps_address(query):
    return gmaps.places_autocomplete(query, types=["geocode"], components={"country": "ca"})


def get_address_info(top_suggestion, unit_number=None):
    coords = geocoords(gmaps.find_place(top_suggestion, 'textquery')['candidates'][0]['place_id'])
    return {
        'description': top_suggestion['description'],
        'street_num': top_suggestion['terms'][0]['value'],
        'street_name': top_suggestion['terms'][1]['value'].split(' ')[0],
        'city': top_suggestion['terms'][2]['value'],
        'unit_number': unit_number,
        'lat': coords['lat'],
        'lon': coords['lon']
    }

In [73]:
def str_to_bool(s):
    if s.lower() == 'true':
         return True
    elif s.lower() == 'false':
         return False
    else:
         raise ValueError("Cannot convert to boolean: " + s)


def price_home_and_find_comparables(address: str, house_category: str,  unit_number:str='', beds:str='', baths:str='', house_sqft:str='', land_sqft:str='', use_comparables:str="false"):
    """
    Price a home and optionally find comparable properties.   
    :param address: The address of the property (e.g. 123 Harvie Ave Toronto, Ontario)
    :param house_category: The type of the house <<Condo, Detached, Semi, Link, Multiplex, Vacant Land, Condo Townhouse, Freehold Townhouse, Other>>
    :param unit_number: The unit number like 402 (optional)
    :param beds: The number of bedrooms (optional)
    :param baths: The number of bathrooms (optional)
    :param house_sqft: The square footage of the house (optional)
    :param land_sqft: The square footage of the land (optional)
    :param use_comparables: Whether to use comparables or not (optional). If set to true, you MUST specify at least one of beds, baths, house_sqft, land_sqft. <<true, false>>

    :return: Pricing information for the property
    """
    print(address, house_category,  unit_number, beds, baths, house_sqft, land_sqft, use_comparables)
    headers = {
        'Authorization': os.getenv("HUGGING_FACE_API_KEY"),
        'Content-Type': 'application/json'
    }
    if(str_to_bool(use_comparables) and (beds == '' and baths == '' and house_sqft == '' and land_sqft == '')):
        return 'You must specify at least one of beds, baths, house_sqft, land_sqft if you want to use comparables'
    data = [
            address,
            house_category,
            unit_number,
            str_to_bool(use_comparables),
            True,
            beds,
            baths,
            house_sqft,
            land_sqft,
            '',
            {'headers': ['None'], 'data': [['None']]}
        ]
    print(data)
    #print({'headers': ['None'], 'data': [['None']]} if modifications is None else modifications)
    response = requests.post("https://rshrott-smartbids.hf.space/run/price_with_google_maps", json={
        "data": data
    }, headers=headers)


    if response.status_code == 200:
        data = response.json()["data"]
        return data
    else:
        return 'Status Code:' + str(response.status_code) + ' ' + str(response.json())
    
json_data1 = price_home_and_find_comparables('218 Harvie Ave Toronto','Detached', '',  '4', '', '', '', "true")
print(json_data1)

218 Harvie Ave Toronto Detached  4    true
['218 Harvie Ave Toronto', 'Detached', '', True, True, '4', '', '', '', '', {'headers': ['None'], 'data': [['None']]}]
[{'148 Hope St': {'exact_match': True, 'weight': 0.2, 'total_beds': 4.0, 'house-washroom': 2.0, 'house-house_area-estimate': None, 'house-land-area': 2035.2, 'prices': {'2023-06-25': {'price_complex': 1161034.987941889, 'price_simple': 1227738.2833422003, 'px': 1227738.2833422003, 'interest_rate': 5.55249999999903}}, 'house-list_dates-date_start': '2010-03-03T00:00:00'}, '164 Harvie Ave': {'exact_match': True, 'weight': 0.2, 'total_beds': 4.0, 'house-washroom': 2.0, 'house-house_area-estimate': None, 'house-land-area': 3072.0, 'prices': {'2023-06-25': {'price_complex': 1197162.5103505768, 'price_simple': 1202894.664947643, 'px': 1202894.664947643, 'interest_rate': 5.55249999999903}}, 'house-list_dates-date_start': '2006-05-22T00:00:00'}, '241 Nairn Ave': {'exact_match': True, 'weight': 0.2, 'total_beds': 4.0, 'house-washroom':

In [74]:
for data in json_data1:
    for key, value in data.items():
        print(key, value)

148 Hope St {'exact_match': True, 'weight': 0.2, 'total_beds': 4.0, 'house-washroom': 2.0, 'house-house_area-estimate': None, 'house-land-area': 2035.2, 'prices': {'2023-06-25': {'price_complex': 1161034.987941889, 'price_simple': 1227738.2833422003, 'px': 1227738.2833422003, 'interest_rate': 5.55249999999903}}, 'house-list_dates-date_start': '2010-03-03T00:00:00'}
164 Harvie Ave {'exact_match': True, 'weight': 0.2, 'total_beds': 4.0, 'house-washroom': 2.0, 'house-house_area-estimate': None, 'house-land-area': 3072.0, 'prices': {'2023-06-25': {'price_complex': 1197162.5103505768, 'price_simple': 1202894.664947643, 'px': 1202894.664947643, 'interest_rate': 5.55249999999903}}, 'house-list_dates-date_start': '2006-05-22T00:00:00'}
241 Nairn Ave {'exact_match': True, 'weight': 0.2, 'total_beds': 4.0, 'house-washroom': 3.0, 'house-house_area-estimate': None, 'house-land-area': 1830.9399999999998, 'prices': {'2023-06-25': {'price_complex': 1130283.6637069867, 'price_simple': 1230120.59370211

In [77]:
json_data1

[{'148 Hope St': {'exact_match': True,
   'weight': 0.2,
   'total_beds': 4.0,
   'house-washroom': 2.0,
   'house-house_area-estimate': None,
   'house-land-area': 2035.2,
   'prices': {'2023-06-25': {'price_complex': 1161034.987941889,
     'price_simple': 1227738.2833422003,
     'px': 1227738.2833422003,
     'interest_rate': 5.55249999999903}},
   'house-list_dates-date_start': '2010-03-03T00:00:00'},
  '164 Harvie Ave': {'exact_match': True,
   'weight': 0.2,
   'total_beds': 4.0,
   'house-washroom': 2.0,
   'house-house_area-estimate': None,
   'house-land-area': 3072.0,
   'prices': {'2023-06-25': {'price_complex': 1197162.5103505768,
     'price_simple': 1202894.664947643,
     'px': 1202894.664947643,
     'interest_rate': 5.55249999999903}},
   'house-list_dates-date_start': '2006-05-22T00:00:00'},
  '241 Nairn Ave': {'exact_match': True,
   'weight': 0.2,
   'total_beds': 4.0,
   'house-washroom': 3.0,
   'house-house_area-estimate': None,
   'house-land-area': 1830.939999

In [90]:
def compute_weighted_avg(json_data):
    total_weight = 0
    weighted_sum = 0
    house_data = []
    comparable_properties = []

    exact_matches = 0
    for data in json_data:
        for key, value in data.items():
            if key == "input_info": # Ignore "input_info" data
                input_info = value
                continue

            weight = value['weight']
            first_price = value['prices'][next(iter(value['prices']))]['price_simple']
            beds = value['total_beds']
            washrooms = value['house-washroom']
            area_estimate = value['house-house_area-estimate']
            land_area = value['house-land-area']
            exact_match = value['exact_match']

            house_data.append({
                "street": key,
                "price": first_price,
                "beds": beds,
                "washrooms": washrooms,
                "area_estimate": area_estimate,
                "land_area": land_area,
                "exact_match": exact_match,
            })

            total_weight += weight
            weighted_sum += weight * first_price

            if exact_match:
                exact_matches += 1

    weighted_avg = weighted_sum / total_weight if total_weight > 0 else 0
    location_coords = (input_info['lat'], input_info['lon'])

    streets = ', '.join([house['street'] for house in house_data])
    for house in house_data:
        comparable_properties.append(f"{house['street']},{house['price']:.2f},{house['beds']},{house['washrooms']},{house['area_estimate']},{house['land_area']}")
    comparable_properties_csv = "\n".join(comparable_properties)

    if len(house_data) == 1 and exact_matches == 1:  # Only one exact match found
        house = house_data[0]
        result_str = f"The model price of {input_info['street_num']} {input_info['street_name']} is {house['price']:.2f}. The location coordinates are {location_coords}. The house has {house['beds']} beds, {house['washrooms']} washrooms, an estimated area of {house['area_estimate']}, and a land area of {house['land_area']}. We priced this with an exact match."
    elif exact_matches == len(house_data): # All houses are exact matches
        result_str = f"The model price of {input_info['street_num']} {input_info['street_name']} is {weighted_avg:.2f}. The location coordinates are {location_coords}. The house has {house['beds']} beds, {house['washrooms']} washrooms, an estimated area of {house['area_estimate']}, and a land area of {house['land_area']}. We priced this with an exact match. You could also price this property with comparables: {streets}.\nThe comparable properties used are (in CSV format):\nStreet,Price,Beds,Washrooms,Area Estimate,Land Area\n{comparable_properties_csv}"
    else: # Not all houses are exact matches, we have comparables
        result_str = f"The model price of {input_info['street_num']} {input_info['street_name']} is {weighted_avg:.2f}. The location coordinates are {location_coords}.This price was computed using comparable properties: {streets}. \nThe comparable properties used are (in CSV format):\nStreet,Price,Beds,Washrooms,Area Estimate,Land Area\n{comparable_properties_csv}"

    return result_str





# Load json_data
json_data1 = price_home_and_find_comparables('218 Harvie Ave Toronto','Detached', '',  '4', '', '', '', "true")

compute_weighted_avg(json_data1)


218 Harvie Ave Toronto Detached  4    true
['218 Harvie Ave Toronto', 'Detached', '', True, True, '4', '', '', '', '', {'headers': ['None'], 'data': [['None']]}]


'The model price of 218 Harvie is 1174830.63. The location coordinates are (43.6817537, -79.4523143). The house has 4.0 beds, 2.0 washrooms, an estimated area of None, and a land area of 2304.0. We priced this with an exact match. You could also price this property with comparables: 148 Hope St, 164 Harvie Ave, 241 Nairn Ave, 141 Harvie Ave, 205 Nairn Ave.\nThe comparable properties used are (in CSV format):\nStreet,Price,Beds,Washrooms,Area Estimate,Land Area\n148 Hope St,1227738.28,4.0,2.0,None,2035.2\n164 Harvie Ave,1202894.66,4.0,2.0,None,3072.0\n241 Nairn Ave,1230120.59,4.0,3.0,None,1830.9399999999998\n141 Harvie Ave,1108656.19,4.0,3.0,1300.0,2432.0\n205 Nairn Ave,1104743.42,4.0,2.0,None,2304.0'

In [91]:
json_data1 = price_home_and_find_comparables('218 Harvie Ave Toronto','Detached', '',  '4', '', '', '', "false")

compute_weighted_avg(json_data1)


218 Harvie Ave Toronto Detached  4    false
['218 Harvie Ave Toronto', 'Detached', '', False, True, '4', '', '', '', '', {'headers': ['None'], 'data': [['None']]}]


'The model price of 218 Harvie is 1332436.04. The location coordinates are (43.6817537, -79.4523143). The house has 3.5 beds, 4.0 washrooms, an estimated area of 1917.0, and a land area of 2286.0. We priced this with an exact match.'

: 

In [51]:
compute_weighted_avg(json_data2)

'The model price of 218 Harvie is 1332436.04. The house has 3.5 beds, 4.0 washrooms, an estimated area of 1917.0, and a land area of 2286.0. We priced this with an exact match. The location coordinates are (43.6817537, -79.4523143).'

In [53]:
    
json_data2 = price_home_and_find_comparables('218 Harvie Ave Toronto','Detached', '',  '4', '', '', '', "true")
for data in json_data2:
    for key, value in data.items():
        print(key, value)

218 Harvie Ave Toronto Detached  4    true
['218 Harvie Ave Toronto', 'Detached', '', True, True, '4', '', '', '', '', {'headers': ['None'], 'data': [['None']]}]
148 Hope St {'exact_match': True, 'weight': 0.2, 'total_beds': 4.0, 'house-washroom': 2.0, 'house-house_area-estimate': None, 'house-land-area': 2035.2, 'prices': {'2023-06-25': {'price_complex': 1161034.987941889, 'price_simple': 1227738.2833422003, 'px': 1227738.2833422003, 'interest_rate': 5.55249999999903}}, 'house-list_dates-date_start': '2010-03-03T00:00:00'}
164 Harvie Ave {'exact_match': True, 'weight': 0.2, 'total_beds': 4.0, 'house-washroom': 2.0, 'house-house_area-estimate': None, 'house-land-area': 3072.0, 'prices': {'2023-06-25': {'price_complex': 1197162.5103505768, 'price_simple': 1202894.664947643, 'px': 1202894.664947643, 'interest_rate': 5.55249999999903}}, 'house-list_dates-date_start': '2006-05-22T00:00:00'}
241 Nairn Ave {'exact_match': True, 'weight': 0.2, 'total_beds': 4.0, 'house-washroom': 3.0, 'house-

In [69]:
def get_gmaps_address(query):
    return gmaps.places_autocomplete(query, types=["geocode"], components={"country": "ca"})

def get_gmaps_address(query):
    suggestions = gmaps.places_autocomplete(query, types=["geocode"], components={"country": "ca"})
    return [s for s in suggestions if any(x['value'].lower() in ['on', 'bc', 'ab', "ontario", "british columbia", "alberta"]  for x in s['terms'])]
print(get_gmaps_address('218 Harvie Ave Toronto Ontario'))
print(get_gmaps_address('218 Harvie Ave Toronto Ontario'))



[{'description': '218 Harvie Ave, Toronto, Ontario, Canada', 'matched_substrings': [{'length': 3, 'offset': 0}, {'length': 10, 'offset': 4}, {'length': 7, 'offset': 16}, {'length': 7, 'offset': 25}], 'place_id': 'ChIJkcElyws0K4gRWfSoUnq_-BM', 'reference': 'ChIJkcElyws0K4gRWfSoUnq_-BM', 'structured_formatting': {'main_text': '218 Harvie Ave', 'main_text_matched_substrings': [{'length': 3, 'offset': 0}, {'length': 10, 'offset': 4}], 'secondary_text': 'Toronto, Ontario, Canada', 'secondary_text_matched_substrings': [{'length': 7, 'offset': 0}, {'length': 7, 'offset': 9}]}, 'terms': [{'offset': 0, 'value': '218'}, {'offset': 4, 'value': 'Harvie Ave'}, {'offset': 16, 'value': 'Toronto'}, {'offset': 25, 'value': 'Ontario'}, {'offset': 34, 'value': 'Canada'}], 'types': ['premise', 'geocode']}]
[{'description': '218 Harvie Ave, Toronto, Ontario, Canada', 'matched_substrings': [{'length': 3, 'offset': 0}, {'length': 10, 'offset': 4}, {'length': 7, 'offset': 16}, {'length': 7, 'offset': 25}], 'p

In [56]:
get_gmaps_address('218 Harvie Ave Toronto Ontario')

[{'description': '218 Harvie Ave, Toronto, Ontario, Canada',
  'matched_substrings': [{'length': 3, 'offset': 0},
   {'length': 10, 'offset': 4},
   {'length': 7, 'offset': 16},
   {'length': 7, 'offset': 25}],
  'place_id': 'ChIJkcElyws0K4gRWfSoUnq_-BM',
  'reference': 'ChIJkcElyws0K4gRWfSoUnq_-BM',
  'structured_formatting': {'main_text': '218 Harvie Ave',
   'main_text_matched_substrings': [{'length': 3, 'offset': 0},
    {'length': 10, 'offset': 4}],
   'secondary_text': 'Toronto, Ontario, Canada',
   'secondary_text_matched_substrings': [{'length': 7, 'offset': 0},
    {'length': 7, 'offset': 9}]},
  'terms': [{'offset': 0, 'value': '218'},
   {'offset': 4, 'value': 'Harvie Ave'},
   {'offset': 16, 'value': 'Toronto'},
   {'offset': 25, 'value': 'Ontario'},
   {'offset': 34, 'value': 'Canada'}],
  'types': ['premise', 'geocode']}]

In [57]:
def get_suggestions(address):
    suggestions = get_gmaps_address(address)
    return suggestions
get_suggestions('218 Harvie Ave Toronto')

[{'description': '218 Harvie Ave, Toronto, ON, Canada',
  'matched_substrings': [{'length': 3, 'offset': 0},
   {'length': 10, 'offset': 4},
   {'length': 7, 'offset': 16}],
  'place_id': 'ChIJkcElyws0K4gRWfSoUnq_-BM',
  'reference': 'ChIJkcElyws0K4gRWfSoUnq_-BM',
  'structured_formatting': {'main_text': '218 Harvie Ave',
   'main_text_matched_substrings': [{'length': 3, 'offset': 0},
    {'length': 10, 'offset': 4}],
   'secondary_text': 'Toronto, ON, Canada',
   'secondary_text_matched_substrings': [{'length': 7, 'offset': 0}]},
  'terms': [{'offset': 0, 'value': '218'},
   {'offset': 4, 'value': 'Harvie Ave'},
   {'offset': 16, 'value': 'Toronto'},
   {'offset': 25, 'value': 'ON'},
   {'offset': 29, 'value': 'Canada'}],
  'types': ['premise', 'geocode']}]

In [58]:
get_suggestions('218 Harvie Ave Toronto, ontario')

[{'description': '218 Harvie Ave, Toronto, Ontario, Canada',
  'matched_substrings': [{'length': 3, 'offset': 0},
   {'length': 10, 'offset': 4},
   {'length': 7, 'offset': 16},
   {'length': 7, 'offset': 25}],
  'place_id': 'ChIJkcElyws0K4gRWfSoUnq_-BM',
  'reference': 'ChIJkcElyws0K4gRWfSoUnq_-BM',
  'structured_formatting': {'main_text': '218 Harvie Ave',
   'main_text_matched_substrings': [{'length': 3, 'offset': 0},
    {'length': 10, 'offset': 4}],
   'secondary_text': 'Toronto, Ontario, Canada',
   'secondary_text_matched_substrings': [{'length': 7, 'offset': 0},
    {'length': 7, 'offset': 9}]},
  'terms': [{'offset': 0, 'value': '218'},
   {'offset': 4, 'value': 'Harvie Ave'},
   {'offset': 16, 'value': 'Toronto'},
   {'offset': 25, 'value': 'Ontario'},
   {'offset': 34, 'value': 'Canada'}],
  'types': ['premise', 'geocode']}]

In [63]:
suggestion = get_suggestions('218 Harvie Ave, Toronto, Ontario')
print(suggestion)
get_address_info(suggestion[0], '')

[{'description': '218 Harvie Ave, Toronto, Ontario, Canada', 'matched_substrings': [{'length': 3, 'offset': 0}, {'length': 10, 'offset': 4}, {'length': 7, 'offset': 16}, {'length': 7, 'offset': 25}], 'place_id': 'ChIJkcElyws0K4gRWfSoUnq_-BM', 'reference': 'ChIJkcElyws0K4gRWfSoUnq_-BM', 'structured_formatting': {'main_text': '218 Harvie Ave', 'main_text_matched_substrings': [{'length': 3, 'offset': 0}, {'length': 10, 'offset': 4}], 'secondary_text': 'Toronto, Ontario, Canada', 'secondary_text_matched_substrings': [{'length': 7, 'offset': 0}, {'length': 7, 'offset': 9}]}, 'terms': [{'offset': 0, 'value': '218'}, {'offset': 4, 'value': 'Harvie Ave'}, {'offset': 16, 'value': 'Toronto'}, {'offset': 25, 'value': 'Ontario'}, {'offset': 34, 'value': 'Canada'}], 'types': ['premise', 'geocode']}]


{'description': '218 Harvie Ave, Toronto, Ontario, Canada',
 'street_num': '218',
 'street_name': 'Harvie',
 'city': 'Toronto',
 'unit_number': '',
 'lat': 43.6817537,
 'lon': -79.4523143}

In [3]:
from airtable import Airtable
from pydantic import BaseModel, Field
from typing import Optional
import os
from dotenv import load_dotenv
print('pattd09wj9GOuUjQL.a1c86dbc5172baeb10898109578e171fe5b5b59ce7437abb08405057b90f7203')
load_dotenv(".env")
print(os.environ['AIRTABLE_BASE_ID'], os.environ['AIRTABLE_TABLE_NAME'], os.environ['AIRTABLE_API_KEY'])
airtable = Airtable(os.environ['AIRTABLE_BASE_ID'], os.environ['AIRTABLE_TABLE_NAME'], api_key=os.environ['AIRTABLE_API_KEY'])

pattd09wj9GOuUjQL.a1c86dbc5172baeb10898109578e171fe5b5b59ce7437abb08405057b90f7203
appNHhSJqYGaYrx1t tblS5jdNd3W94kyov pattd09wj9GOuUjQL.a1c86dbc5172baeb10898109578e171fe5b5b59ce7437abb08405057b90f7203


In [4]:
client = airtable.search('email', 'ryanshrott@gmail.com')
client

[{'id': 'reckfOBURRVrWwCMz',
  'createdTime': '2023-07-04T01:38:19.000Z',
  'fields': {'name': 'Ryan Shrott',
   'email': 'ryanshrott@gmail.com',
   'phone': '6479727058',
   'verified': True,
   'token': '26f1fad0dcdc7b15eaa5fbb4b53a5d16e5339132'}}]

In [9]:
def find_client_in_airtable(email):
    airtable = Airtable(os.environ['AIRTABLE_BASE_ID'], os.environ['AIRTABLE_TABLE_NAME'], api_key=os.environ['AIRTABLE_API_KEY'])
    client = airtable.search('email', email)
    return client
find_client_in_airtable('ryanshrott@gmail.com')

[{'id': 'reckfOBURRVrWwCMz',
  'createdTime': '2023-07-04T01:38:19.000Z',
  'fields': {'name': 'Ryan Shrott',
   'email': 'ryanshrott@gmail.com',
   'phone': '6479727058',
   'verified': True,
   'token': '26f1fad0dcdc7b15eaa5fbb4b53a5d16e5339132'}}]